In [1]:
import numpy
import pandas as pd
import nltk
import re
import io

In [2]:
#titles = io.open("titles.txt", mode="r", encoding="utf-8", errors="ignore").read().split('\n')
docs = io.open("docs.txt", mode="r", encoding="utf-8", errors="ignore").read()
docs = re.sub(r"\[\d+\]", "", docs)
docs = re.sub(r'\w*\d\w*', '', docs).split('\n')

titles = [docs[i] for i in range(len(docs)) if i % 2 == 0]
docs = [docs[i] for i in range(len(docs)) if i % 2 == 1]

ranks = []

for i in range(0,len(titles)):
    ranks.append(i)
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

421 titles
421 docs
National Diet Library
b'The National Diet Library (NDL) (\\\xbd\\xab\\\\xbd\\xbc\\\\\\\\\\, Kokuritsu Kokkai Toshokan) is the national library of Japan and among the largest libraries in the world. It was established in  for the purpose of assisting members of the National Diet of Japan (\\\xbd\\xbc\, Kokkai) in researching matters of public policy. The library is similar in purpose and scope to the United States Library of Congress.'b'The National Diet Library (NDL) consists of two main facilities in Tokyo and Kyoto, and several other branch libraries throughout Japan.'b''b''b"The National Diet Library is the successor of three separate libraries: the library of the House of Peers, the library of the House of Representatives, both of which were established at the creation of Japan's Imperial Diet in ; and the Imperial Library, which had been established in  under the jurisdiction of the Ministry of Education."b'The Diet\'s power in prewar Japan was limited, and its

In [3]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

totalvocab_stemmed = []
totalvocab_tokenized = []
for i in docs:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [4]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.1, stop_words= stopwords,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(docs)

print(tfidf_matrix.shape)

CPU times: user 24.8 s, sys: 351 ms, total: 25.1 s
Wall time: 25.7 s
(421, 4964)


In [5]:
terms = tfidf_vectorizer.get_feature_names()

from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf_matrix)

In [16]:
# The first type of dictionary
dict1 = {}
for i in range(len(titles)):
    dict1[titles[i]]= cos_sim[i]
    
for x in dict1:
    print (x)
print(dict1['Elementary algebra'])
print(len(dict1['Elementary algebra']))

Elementary algebra
Euclidean space
Mathematics
Linear equation
Linear map
Matrix (mathematics)
Vector space
Geometry
Line (geometry)
Plane (geometry)
Rotation (mathematics)
Functional analysis
Engineering
Mathematical model
Nonlinear system
Determinant
System of linear equations
Gottfried Wilhelm Leibniz
Gabriel Cramer
Cramer's rule
Carl Friedrich Gauss
Gaussian elimination
Geodesy
Hermann Grassmann
James Joseph Sylvester
Arthur Cayley
Hüseyin Tevfik Pasha
Giuseppe Peano
Abstract algebra
Quantum mechanics
Special relativity
Statistics
Algorithm
School Mathematics Study Group
Secondary school
Singular-value decomposition
Field (mathematics)
Real number
Set (mathematics)
Binary operation
Element (mathematics)
Euclidean vector
Scalar multiplication
Axiom
Associative property
Commutative property
Identity element
Zero element
Inverse element
Additive inverse
Distributive property

Abelian group
Sequence
Function (mathematics)
Polynomial ring
Map (mathematics)
Bijection
Isomorphism
Range (m

In [18]:
# The second type of dictionary
dict = {}
terms_num = len(titles)
for i in range(terms_num):y
    for j in range(terms_num):
        dict[(titles[i],titles[j])] = cos_sim[i][j]
print(dict[('Elementary algebra', 'Number theory')])

0.256569396173


In [ ]:
dict["pages"][1]["title"]
dict["pages"][1]["text"]